In [3]:
import numpy as np
import pandas as pd
import h5pyd
import xarray as xr

In [8]:
#ds = xr.open_dataset('t2m_1985_2024.nc')
#ds= ds.sortby('time')
#ds = ds.drop_vars('t2m')

C:\Users\Caleb Derrickson\AppData\Local\Temp\ipykernel_540\1674457491.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset('t2m_1985_2024.nc')


In [13]:
fname = "temperatures_by_gid_time_phil.h5"
hdf = pd.HDFStore(fname)
map_uid_df = hdf.get('/gids')
temp_df = hdf.get('/temps')

# Specifying the given date range
temp_df = temp_df[temp_df.index.year >= 2025]

# Dropping all duplicates
unique_map_uid_df = map_uid_df.drop_duplicates(subset='gid', keep='first').set_index('gid')
unique_map_uid_df = unique_map_uid_df.loc[temp_df.columns]

In [21]:
#map_uid_df = map_uid_df.set_index('gid').loc[temp_df.columns]

# Assuming unique_map_uid_df is already processed as before:
# unique_map_uid_df = map_uid_df.drop_duplicates(subset='gid').set_index('gid')
# unique_map_uid_df = unique_map_uid_df.loc[temp_df.columns]

# Now create the xarray Dataset
dS = xr.Dataset(
    data_vars={
        'temp': (['time', 'gid'], temp_df.values)
    },
    coords={
        'time': pd.to_datetime(temp_df.index),
        'gid': temp_df.columns,
        'latitude': (['gid'], unique_map_uid_df['latitude'].values),
        'longitude': (['gid'], unique_map_uid_df['longitude'].values)
    }
)

# converting times to datetimeindex
dS['time'] = pd.DatetimeIndex(dS['time'].values)

In [22]:
dS

<xarray.Dataset> Size: 2MB
Dimensions:    (time: 43824, gid: 8)
Coordinates:
  * time       (time) datetime64[ns] 351kB 2025-01-01 ... 2029-12-31T23:00:00
  * gid        (gid) int64 64B 1656925 1636340 1410985 ... 1595100 1781383
    latitude   (gid) float64 64B 31.0 31.25 34.0 32.0 29.75 27.75 31.75 29.5
    longitude  (gid) float64 64B -103.0 -99.75 -98.5 ... -98.25 -95.25 -95.75
Data variables:
    temp       (time, gid) float32 1MB 0.86 0.14 -1.28 ... 23.79 10.96 15.63

In [20]:
dS.to_netcdf("sup3rcc_t2m_2025_2030.nc")